# post-processing

Scratch code to post-process `analysis.py` log files.

**TODO:**
- automate post-processing to compare centralized vs decentralized
- consider logging stuff as dicts with eval instead of lists
- move to hardware

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import decentralized as dec

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport decentralized.util

In [ ]:
%matplotlib ipympl
plt.rcParams['axes.grid'] = True

In [ ]:
fig = plt.figure()

In [ ]:
filename = dec.repopath / "logs" / "dec-mc-2_08-03-22_18.36.15.csv"
df = pd.read_csv(filename)

In [ ]:
# Turn string lists into actual lists.
df.loc[:, 'ids'] = df.loc[:, 'ids'].apply(eval)
df.loc[:, 'times'] = df.loc[:, 'times'].apply(eval)